# Out Of Tree (OOT) Examples

This notebook demonstrates how to define "Out Of Tree" parameters and outputs. That is, parameters and outputs other than those defined inside the main framework.

In [ ]:
import Py6S
import alive_progress
import numpy as np

import rtm_wrapper.parameters as rtm_param
import rtm_wrapper.plot as rtm_plot
from rtm_wrapper.engines.sixs import PySixSEngine, pysixs_default_inputs
from rtm_wrapper.execution import ConcurrentExecutor, ParallelConcurrentExecutor
from rtm_wrapper.simulation import SweepSimulation

## Define OOT input parameter

In [ ]:
class Frequency(rtm_param.Parameter):
    frequency = rtm_param.FloatField(title="Frequency", unit="terahertz")


@PySixSEngine.params.register("wavelength")
def handle_frequency(inputs: Frequency, wrapper: Py6S.SixS) -> None:
    micrometers = 1e6 * ((3e8 / 1e12) / inputs.frequency)
    wrapper.wavelength = Py6S.Wavelength(micrometers)

## Simulation using OOT input parameter

In [ ]:
sweep = SweepSimulation(
    {
        "wavelength.frequency": np.linspace(200, 600, 100),
    },
    base=pysixs_default_inputs().replace(wavelength=Frequency())
)


engine = PySixSEngine()
runner = ConcurrentExecutor()

with alive_progress.alive_bar(runner.steps_for(sweep), force_tty=True) as bar:
    runner.run(sweep, engine, step_callback=lambda _: bar())

results = runner.collect_results()

fig, ax = rtm_plot.plot_sweep_single(results.total_transmission)
ax.set_title("Custom Parameter Example")
ax.grid(True)
display(results)

## Define OOT output

In [ ]:
@PySixSEngine.outputs.register(title="Half Total Transmission", unit="1")
def half_total_transmission(total_transmission: float) -> float:
    return total_transmission / 2 

## Simulation with OOT output

In [ ]:
sweep = SweepSimulation(
    {
        "wavelength.frequency": np.linspace(100, 600, 100),
    },
    base=pysixs_default_inputs().replace(wavelength=Frequency())
)

engine = PySixSEngine(outputs=["half_total_transmission"])
runner = ConcurrentExecutor()

with alive_progress.alive_bar(runner.steps_for(sweep), force_tty=True) as bar:
    runner.run(sweep, engine, step_callback=lambda _: bar())

results = runner.collect_results()

fig, ax = rtm_plot.plot_sweep_single(results.half_total_transmission)
ax.set_title("Custom Output Example")
ax.grid(True)
display(results)